In [ ]:
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

#ML related libraries
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Reading data & data pre-processing

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df.head()

Lets check how many NaNs we have:

In [ ]:
df.isna().sum()

In [ ]:
df.shape

It is better to drop the following columns as they have almost the same number of NaNs as the deta set shape

In [ ]:
df.drop(columns=['MiscFeature','Fence','PoolQC','FireplaceQu','Alley'],inplace=True)

In [ ]:
df.isna().sum()

Now we remove the remaining rows which contain NaNs

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

lets check data types:

In [ ]:
df.dtypes

# Helper Functions:

In [ ]:
def mean(col): #function that aggeregates and shows the means
    return df.groupby(col).mean().transpose()

In [ ]:
def count(col): #function to count the number of examples for each desired category
    return df[col].value_counts()

In [ ]:
def joint(col1,col2): #function to visualise the joint plot
    ax=sns.jointplot(x=col1, y=col2, data=df, kind="kde")
    return ax

In [ ]:
def correlation(df,size1,size2,cmap): #function to create the correlation matrix
    corr=df.corr()
    plt.figure(figsize=(size1,size2))
    ax=sns.heatmap(corr,vmin=-1,vmax=1,cmap=cmap,annot=True)
    return ax

In [ ]:
def correlation_remove(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in dataset.columns:
                    del dataset[colname] # deleting the column from the dataset

In [ ]:
def dist_plot(df,col):
    plt.figure(figsize=(15,10))
    plt.tight_layout()
    return sns.distplot(df[col])

In [ ]:
def count_plot(col,hue):
    return sns.countplot(x=col, hue=hue, data=df)

In [ ]:
def cat_plot(colx,hue,col,h,asp):
    return sns.catplot(x=colx, hue=hue, col=col,data=df, kind="count",height=h, aspect=asp)

# Some Statistics

In [ ]:
df.describe()

In [ ]:
mean('LotShape')

In [ ]:
mean('Neighborhood')

RoofStyle=Hip has the highest SalePrice mean.

In [ ]:
mean('RoofStyle')

Lot price and Sale price does not seem to have a positive correlation

In [ ]:
joint('LotArea','SalePrice')

Garage Area and Sale Price show some positive correlation. In addition, we have most examples around garage area equal to 200 and 600 and price renge less than 200000

In [ ]:
joint('GarageArea','SalePrice')

Year built show a very small positive correlation with price. However, we can conclude that newly built properties have higher prices. Most properties are built between 1950-1975 and around 2000.

In [ ]:
joint('YearBuilt','SalePrice')

In [ ]:
correlation(df,50,50, 'coolwarm')

from the above correlation matrix we can see that sale price has high correlation with OverallQual, TotalBsmtSF, garage area and garage cars. We can also see that sale price has a negaive correlation with enclosed porch. Let's visualise the distributions of the sale price and its highly corelated columns:

In [ ]:
dist_plot(df,'SalePrice')

The price distribution is skewed to left.

In [ ]:
dist_plot(df,'GarageArea')

In [ ]:
dist_plot(df,'OverallQual')

In [ ]:
dist_plot(df,'TotalBsmtSF')

In [ ]:
dist_plot(df,'GarageCars')

In [ ]:
dist_plot(df,'EnclosedPorch')

In [ ]:
count('SaleCondition')

Most properties are in normal conditions. at the second we have partial with 119 properties

# Machine Learning

let's get a list of all categorical columns:

In [ ]:
list_cat=df.select_dtypes(include=['object', 'category']).columns

In [ ]:
df=pd.get_dummies(data=df, columns=list_cat)

In [ ]:
X=df[['SalePrice']]
Y=df.loc[:, df.columns != 'SalePrice']

Let's split the data to train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
regressor = LinearRegression(normalize=True)  
regressor.fit(X_train, y_train) #training the algorithm

Let's check the regressor parameters:

In [ ]:
# intercept:
print(regressor.intercept_)
# slope:
print(regressor.coef_)

Lets predict the values for the test data set. 

In [ ]:
y_pred = regressor.predict(X_test)

Lets check the metrics

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root mean square shows that our model is not very accurate however it provides some insights how sale price would change with multiple variables we have in the data set. This is mainly because the data set is non-linear. We can use various non-linear models such as random forest regressor or deep learning such as RNN/LSTM to build a more accurate model. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg_rfc=RandomForestRegressor(n_estimators=10,random_state=0)
reg_rfc.fit(X_train,y_train)

In [ ]:
pred=reg_rfc.predict(X_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

It seems that all metrics increased using random forest! We might need to do some grid search to tune hyper parameters. Due to the lack of time, I just ignore doing hyperparameters tuning and assume that linear regression performed better than random forest regressor!

# Prediction for Test dataset

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test.drop(columns=['MiscFeature','Fence','PoolQC','FireplaceQu','Alley'],inplace=True)

In [ ]:
test.fillna(0,inplace=True)

In [ ]:
list_cat=test.select_dtypes(include=['object', 'category']).columns

In [ ]:
test=pd.get_dummies(data=test, columns=list_cat)

Now we need to check if we have the same dummy variables in test data set as the ones in the train data set

In [ ]:
def add_missing_dummy_columns( df, columns ):
    missing_cols = set( columns ) - set( df.columns )
    for c in missing_cols:
        df[c] = 0

In [ ]:
def fix_columns( df, columns ):  

    add_missing_dummy_columns( df, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( df.columns ) == set())

    extra_cols = set( df.columns ) - set( columns )
    if extra_cols:
        print ("extra columns:", extra_cols)

    df = df[ columns ]
    return df.head()

In [269]:
fix_columns(test,df.columns)

extra columns: {'GarageCond_0', 'BsmtFinType2_0', 'GarageFinish_0', 'GarageType_0', 'MSZoning_0', 'Functional_0', 'MasVnrType_0', 'Exterior1st_AsphShn', 'Foundation_Slab', 'BsmtFinType1_0', 'Exterior2nd_0', 'BsmtExposure_0', 'Exterior1st_0', 'Functional_Sev', 'RoofStyle_Shed', 'ExterCond_Po', 'BsmtQual_0', 'GarageQual_0', 'BsmtCond_0', 'KitchenQual_0', 'SaleType_0', 'Heating_Wall', 'Utilities_0'}


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_A

In [ ]:
test_pred=regressor.predict('test')